<a href="https://colab.research.google.com/github/imnaimur/cse428/blob/main/project/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [36]:
import os
import numpy as np
import matplotlib.pyplot as plt
import skimage
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets,transforms
from pathlib import Path
from PIL import Image

##Data preperations

In [65]:
# Install kagglehub
# !pip install kagglehub

import kagglehub

# Download latest version
path = kagglehub.dataset_download("briscdataset/brisc2025")
path = os.path.join(path,"brisc2025")


In [61]:
print("Downloaded dataset path:", path)
print("Contents:", os.listdir(path))
import os

Downloaded dataset path: /kaggle/input/brisc2025
Contents: ['brisc2025']


###Path setup

In [57]:
base_dir = "/content/drive/MyDrive/brisc2025"

In [63]:
import os
print("Inside base_dir:", os.listdir(path))


Inside base_dir: ['brisc2025']


###Segmentations data class

In [33]:


class BrainSegmentationDataset(Dataset):
    def __init__(self, images_dir, masks_dir, transform=None, target_transform=None):
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.transform = transform
        self.target_transform = target_transform

        self.images = sorted(os.listdir(images_dir))
        self.masks = sorted(os.listdir(masks_dir))

        assert len(self.images) == len(self.masks), "Number of images and masks must match!"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.images_dir, self.images[idx])
        mask_path = os.path.join(self.masks_dir, self.masks[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # grayscale mask

        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            mask = self.target_transform(mask)

        return image, mask


###classification data func

In [34]:
def get_classification_dataset(root_dir, transform=None):
    """
    Loads classification dataset where images are stored in class subfolders.
    Example:
        root_dir/train/glioma, meningioma, pituitary, no_tumor
    """
    return datasets.ImageFolder(root=root_dir, transform=transform)


In [68]:
# --- transforms ---
transform_img = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.ToTensor()
])

transform_mask = transforms.Compose([
    transforms.Resize((600, 600)),
    transforms.ToTensor()
])

# --- Segmentation Train Loader ---
seg_train = BrainSegmentationDataset(
    images_dir=os.path.join(path, "segmentation_task/train/images"),
    masks_dir=os.path.join(path, "segmentation_task/train/masks"),
    transform=transform_img,
    target_transform=transform_mask
)
seg_train_loader = DataLoader(seg_train, batch_size=16, shuffle=True)

# --- Classification Train Loader ---
cls_train = get_classification_dataset(
    root_dir=os.path.join(path, "classification_task/train"), # Corrected path
    transform=transform_img
)
cls_train_loader = DataLoader(cls_train, batch_size=16, shuffle=True)

# --- Check ---
imgs, masks = next(iter(seg_train_loader))
print("Segmentation ->", imgs.shape, masks.shape)

cls_imgs, cls_labels = next(iter(cls_train_loader))
print("Classification ->", cls_imgs.shape, cls_labels.shape)
print("Class mapping:", cls_train.classes)

Segmentation -> torch.Size([16, 3, 600, 600]) torch.Size([16, 1, 600, 600])
Classification -> torch.Size([16, 3, 600, 600]) torch.Size([16])
Class mapping: ['glioma', 'meningioma', 'no_tumor', 'pituitary']


In [70]:
import random
import matplotlib.pyplot as plt

# Pick random index
idx = random.randint(0, len(cls_train) - 1)
img, label = cls_train[idx]

# Convert tensor to numpy
img_np = img.permute(1, 2, 0).numpy()

plt.imshow(img_np)
plt.title(f"Class: {cls_train.classes[label]}")
plt.axis("off")
plt.show()
